In [1]:
import os
print(os.getcwd())

import sys
path = "/".join(os.getcwd().split("/")[:-2])
if path not in sys.path:
    sys.path.append(path)
print(sys.path)

/Users/yashmakkar/Documents/Project/GEO/src/notebooks
['/Library/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/yashmakkar/Documents/Project/GEO/.venv/lib/python3.12/site-packages', '/Users/yashmakkar/Documents/Project/GEO']


In [13]:
from langchain.chat_models import init_chat_model
from src.utils.config import config
import requests
from bs4 import BeautifulSoup

In [6]:
openai_llm = init_chat_model(model="openai:gpt-4o-mini", api_key=config.openai_api_key)
google_llm = init_chat_model("gemini-2.5-flash-preview-05-20", temperature = 0, model_provider="google_genai", api_key=config.google_api_key)

In [9]:
url = "https://www.xtandi.com"
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.6 Safari/605.1.15",
}

response = requests.get(url=url, headers=headers)

In [14]:
page_content = BeautifulSoup(response.content, "html.parser")


In [21]:
page_text = []
for line in page_content.text.split("\n"):
    if len(line) > 0:
        page_text.append(line)

page_text = "\n".join(page_text)

In [27]:
query_generation_prompt = f"""Given the following website content, generate a diverse set of user queries that humans might type into a search engine to find information from this site. Your queries should include:

- Simple, straightforward questions (e.g., factual lookups, one-step queries)
- Complex, multi-part or exploratory questions that require deeper reasoning or synthesis
- Ensure that the queries are phrased naturally, just as a typical human user would search online.

Output a list of search queries relevant to the content above, clearly showing both normal and complex examples."""

query_gen_input = [
    {
        "role": "system",
        "content": query_generation_prompt
    },
    {
        "role": "user",
        "content": page_text
    }
]

In [28]:
query_gen_input

[{'role': 'system',
  'content': 'Given the following website content, generate a diverse set of user queries that humans might type into a search engine to find information from this site. Your queries should include:\n\n- Simple, straightforward questions (e.g., factual lookups, one-step queries)\n- Complex, multi-part or exploratory questions that require deeper reasoning or synthesis\n- Ensure that the queries are phrased naturally, just as a typical human user would search online.\n\nOutput a list of search queries relevant to the content above, clearly showing both normal and complex examples.'},
 {'role': 'user',
  'content': 'XTANDI® (enzalutamide) | Advanced Prostate Cancer Treatment\nYou are now leaving XTANDI.com\nPlease verify that you are a U.S. Healthcare Professional.\nThis information is intended for U.S. Healthcare Professionals only.\nAre you a U.S. Healthcare Professional?\nNo\nYes\nYou are now leaving XTANDI.com\nThe website you are about to visit is not owned or co

In [30]:
queries = openai_llm.invoke(query_gen_input)

In [37]:
queries

AIMessage(content='1. What is XTANDI used for?\n2. How does XTANDI work in treating prostate cancer?\n3. What are the possible side effects of XTANDI?\n4. Can women take XTANDI?\n5. What should I tell my doctor before starting XTANDI?\n6. How do I take XTANDI?\n7. Where can I find the prescribing information for XTANDI?\n8. What should I do if I miss a dose of XTANDI?\n9. Are there any financial support options available for patients taking XTANDI?\n10. How can I sign up for support emails from XTANDI?\n11. What types of prostate cancer can be treated with XTANDI?\n12. How commonly is XTANDI prescribed compared to other prostate cancer treatments?\n13. What resources are available for patients taking XTANDI?\n14. Are there any restrictions on who can receive information about XTANDI?\n15. What are the risks of seizures while taking XTANDI?\n16. If I have a history of heart disease, is it safe to take XTANDI?\n17. How do the side effects of XTANDI compare to those of other prostate canc

In [38]:
search_queries = queries.content.split("\n")
search_queries = [query.split(".")[1].strip() for query in search_queries]           # To remove numerals from AI response.
search_queries

['What is XTANDI used for?',
 'How does XTANDI work in treating prostate cancer?',
 'What are the possible side effects of XTANDI?',
 'Can women take XTANDI?',
 'What should I tell my doctor before starting XTANDI?',
 'How do I take XTANDI?',
 'Where can I find the prescribing information for XTANDI?',
 'What should I do if I miss a dose of XTANDI?',
 'Are there any financial support options available for patients taking XTANDI?',
 'How can I sign up for support emails from XTANDI?',
 'What types of prostate cancer can be treated with XTANDI?',
 'How commonly is XTANDI prescribed compared to other prostate cancer treatments?',
 'What resources are available for patients taking XTANDI?',
 'Are there any restrictions on who can receive information about XTANDI?',
 'What are the risks of seizures while taking XTANDI?',
 'If I have a history of heart disease, is it safe to take XTANDI?',
 'How do the side effects of XTANDI compare to those of other prostate cancer medications?',
 'What sho

In [39]:
search_queries[0]

'What is XTANDI used for?'

In [45]:
ge_prompt = [
    {
        "role": "system",
        "content": """
        You are ChatGPT, an advanced AI language model with web search capabilities. When a user's question requires recent or factual information, perform a search using the web tool, summarize the findings, and respond conversationally. Always include helpful citations and direct links to sources in your answer. Attribute information clearly, integrating context from both the web and the ongoing chat. Respond naturally and accurately, supporting your statements with retrieved evidence and making references easy to follow.
        """
    },
    {
        "role": "user",
        "content": search_queries[0]
    }
]

ge_prompt

[{'role': 'system',
  'content': "\n        You are ChatGPT, an advanced AI language model with web search capabilities. When a user's question requires recent or factual information, perform a search using the web tool, summarize the findings, and respond conversationally. Always include helpful citations and direct links to sources in your answer. Attribute information clearly, integrating context from both the web and the ongoing chat. Respond naturally and accurately, supporting your statements with retrieved evidence and making references easy to follow.\n        "},
 {'role': 'user', 'content': 'What is XTANDI used for?'}]

In [49]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-4o-mini",
    tools=[{"type": "web_search"}],
    input=ge_prompt,
    tool_choice="required"
)

print(response.output_text)

XTANDI (enzalutamide) is a medication primarily used to treat various forms of prostate cancer. It functions as an androgen receptor inhibitor, blocking the effects of male hormones like testosterone, which can stimulate the growth of prostate cancer cells. ([xtandi.com](https://www.xtandi.com/how-xtandi-works?utm_source=openai))

Specifically, XTANDI is indicated for:

- **Metastatic Castration-Sensitive Prostate Cancer (mCSPC):** Prostate cancer that has spread to other parts of the body but still responds to treatments that lower testosterone levels. ([ema.europa.eu](https://www.ema.europa.eu/en/medicines/human/EPAR/xtandi?utm_source=openai))

- **Non-Metastatic Castration-Resistant Prostate Cancer (nmCRPC):** Prostate cancer that has not spread to other parts of the body but continues to grow despite treatments that lower testosterone levels. ([ema.europa.eu](https://www.ema.europa.eu/en/medicines/human/EPAR/xtandi?utm_source=openai))

- **Metastatic Castration-Resistant Prostate C

In [60]:
citations = response.output[1].content[0].annotations
citations

[AnnotationURLCitation(end_index=331, start_index=258, title='How XTANDI® (enzalutamide) Works', type='url_citation', url='https://www.xtandi.com/how-xtandi-works?utm_source=openai'),
 AnnotationURLCitation(end_index=653, start_index=560, title='Xtandi | European Medicines Agency (EMA)', type='url_citation', url='https://www.ema.europa.eu/en/medicines/human/EPAR/xtandi?utm_source=openai'),
 AnnotationURLCitation(end_index=952, start_index=859, title='Xtandi | European Medicines Agency (EMA)', type='url_citation', url='https://www.ema.europa.eu/en/medicines/human/EPAR/xtandi?utm_source=openai'),
 AnnotationURLCitation(end_index=1254, start_index=1161, title='Xtandi | European Medicines Agency (EMA)', type='url_citation', url='https://www.ema.europa.eu/en/medicines/human/EPAR/xtandi?utm_source=openai'),
 AnnotationURLCitation(end_index=1429, start_index=1356, title='How XTANDI® (enzalutamide) Works', type='url_citation', url='https://www.xtandi.com/how-xtandi-works?utm_source=openai'),
 